In [1]:
from liveF1Wrapper.adapter import LivetimingF1Adapter, LivetimingF1Request, LivetimingF1GetData
from liveF1Wrapper.season import Season
from liveF1Wrapper.functions import *
from liveF1Wrapper.events import *
import urllib
import json
import numpy as np

In [8]:
adapter = LivetimingF1Adapter()
endpoint = "https://livetiming.formula1.com/static/2024/2024-09-01_Italian_Grand_Prix/2024-09-01_Race/DriverRaceInfo.jsonStream"
res_text = adapter.get(endpoint=endpoint)
records = res_text.split('\r\n')[:-1]

tl = 12
data = dict((r[:12], json.loads(r[12:])) for r in records)
data

{'00:00:02.381': {'4': {'RacingNumber': '4',
   'Position': '1',
   'Gap': 'LAP 1',
   'Interval': 'LAP 1',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '81': {'RacingNumber': '81',
   'Position': '2',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '63': {'RacingNumber': '63',
   'Position': '3',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '16': {'RacingNumber': '16',
   'Position': '4',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '55': {'RacingNumber': '55',
   'Position': '5',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '44': {'RacingNumber': '44',
   'Position': '6',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False}

In [9]:
endpoint = "https://livetiming.formula1.com/static/2024/2024-09-01_Italian_Grand_Prix/2024-09-01_Race/DriverRaceInfo.jsonStream"
data = LivetimingF1GetData(endpoint, stream=True)
data

{'00:00:02.381': {'4': {'RacingNumber': '4',
   'Position': '1',
   'Gap': 'LAP 1',
   'Interval': 'LAP 1',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '81': {'RacingNumber': '81',
   'Position': '2',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '63': {'RacingNumber': '63',
   'Position': '3',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '16': {'RacingNumber': '16',
   'Position': '4',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '55': {'RacingNumber': '55',
   'Position': '5',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False},
  '44': {'RacingNumber': '44',
   'Position': '6',
   'Gap': '',
   'Interval': '',
   'PitStops': 0,
   'Catching': 1,
   'OvertakeState': 1,
   'IsOut': False}

In [10]:
# [json.loads(val) for val in list(data.values())]

for val in list(data.values()):
    print(val)
    # json.loads(val)

{'4': {'RacingNumber': '4', 'Position': '1', 'Gap': 'LAP 1', 'Interval': 'LAP 1', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '81': {'RacingNumber': '81', 'Position': '2', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '63': {'RacingNumber': '63', 'Position': '3', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '16': {'RacingNumber': '16', 'Position': '4', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '55': {'RacingNumber': '55', 'Position': '5', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '44': {'RacingNumber': '44', 'Position': '6', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '1': {'RacingNumber': '1', 'Position': '7', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '11': {'Racin

In [17]:
records = []

endpoint = "https://livetiming.formula1.com/static/2024/2024-09-01_Italian_Grand_Prix/2024-09-01_Race/DriverRaceInfo.jsonStream"
data = LivetimingF1GetData(endpoint, stream=True)
data

records = []

for key, value in data.items():
    for driver_no, info in value.items():
        record = {
            **{
                "timestamp": key,
                "DriverNo": driver_no,
            },
            **info
        }
        records.append(record)

    #     if stint:
    #         for pit_count, current_info in stint.items():
    #             record = {
    #                 **{
    #                     "timestamp": key,
    #                     "DriverNo": driver_no,
    #                     "PitCount": pit_count,
    #                 },
    #                 **current_info
    #             }

    #             yield record

pd.DataFrame(records)

,timestamp,DriverNo,RacingNumber,Position,Gap,Interval,PitStops,Catching,OvertakeState,IsOut
0,00:00:02.381,4,4,1,LAP 1,LAP 1,0.0,1.0,1.0,False
1,00:00:02.381,81,81,2,,,0.0,1.0,1.0,False
2,00:00:02.381,63,63,3,,,0.0,1.0,1.0,False
3,00:00:02.381,16,16,4,,,0.0,1.0,1.0,False
4,00:00:02.381,55,55,5,,,0.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...
20828,02:11:49.957,3,NaN,12,+1:33.452,+24.957,NaN,NaN,NaN,NaN
20829,02:11:49.957,43,NaN,13,+1:20.848,+12.771,NaN,NaN,NaN,NaN
20830,02:11:50.960,43,NaN,NaN,+1:21.306,+2.878,NaN,NaN,NaN,NaN
20831,02:11:52.774,3,NaN,13,NaN,+12.144,NaN,NaN,NaN,NaN


In [14]:
for key, value in data.items():
    print(key,value)

00:00:02.381 {'4': {'RacingNumber': '4', 'Position': '1', 'Gap': 'LAP 1', 'Interval': 'LAP 1', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '81': {'RacingNumber': '81', 'Position': '2', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '63': {'RacingNumber': '63', 'Position': '3', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '16': {'RacingNumber': '16', 'Position': '4', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '55': {'RacingNumber': '55', 'Position': '5', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '44': {'RacingNumber': '44', 'Position': '6', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, '1': {'RacingNumber': '1', 'Position': '7', 'Gap': '', 'Interval': '', 'PitStops': 0, 'Catching': 1, 'OvertakeState': 1, 'IsOut': False}, 

In [34]:
records

[]

,timestamp,DriverNo,PitCount,Compound,New,TyresNotChanged,TotalLaps,StartLaps
12,00:49:27.192,1,0,HARD,true,0,0.0,0.0
20,00:57:22.243,1,0,-,-,-,1.0,-
45,00:58:52.249,1,0,-,-,-,2.0,-
65,01:00:17.250,1,0,-,-,-,3.0,-
84,01:01:42.257,1,0,-,-,-,4.0,-
104,01:03:07.272,1,0,-,-,-,5.0,-
126,01:04:32.315,1,0,-,-,-,6.0,-
146,01:05:57.290,1,0,-,-,-,7.0,-
166,01:07:22.306,1,0,-,-,-,8.0,-
185,01:08:47.307,1,0,-,-,-,9.0,-
